### Importing the required libraries

In [1]:
import nltk
from nltk import *
import re

### Required GUI Library

In [2]:
import tkinter as tk
from tkinter import scrolledtext
from tkinter import END

### For reading out summary

In [3]:
import pyttsx3

# Initialize the text-to-speech engine
engine = pyttsx3.init()

In [4]:
def read_sentence():# will read summary
    summary = result_box.get("1.0", "end-1c")
    engine.say(summary)
    engine.runAndWait()

def summarize_text():
    # Get text from the text box
    article_text = text_box.get("1.0", "end-1c")

    # Removing Square Brackets and Extra Spaces, special characters and digits
    article_text = re.sub(r'\[[0-9]*\]', ' ', article_text)
    article_text = re.sub(r'\s+', ' ', article_text)
    formatted_article_text = re.sub('[^a-zA-Z]', ' ', article_text )
    formatted_article_text = re.sub(r'\s+', ' ', formatted_article_text)

    # Converting Tokens to Sentences
    sentence_list = sent_tokenize(article_text)

    # Saving stopwords into the variable stopwords
    stopwords = corpus.stopwords.words('english')

    # Counting word frequencies without stopwords
    word_frequencies = {}
    for word in nltk.word_tokenize(formatted_article_text):
        if word not in stopwords:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

    # Finding the weighted frequency
    maximum_frequncy = max(word_frequencies.values())

    for word in word_frequencies.keys():
        word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
    
    # Calculating Sentence Scores
    sentence_scores = {}
    for sent in sentence_list:
        for word in nltk.word_tokenize(sent.lower()):
            if word in word_frequencies.keys():
                if len(sent.split(' ')) < 35:
                    if sent not in sentence_scores.keys():
                        sentence_scores[sent] = word_frequencies[word]
                    else:
                        sentence_scores[sent] += word_frequencies[word]

    # Generating Summary
    import heapq
    summary_sentences = heapq.nlargest(7, sentence_scores, key=sentence_scores.get)
    summary = ' '.join(summary_sentences)           

    # Display summarized text in the result box
    result_box.delete(1.0, END)
    result_box.insert(END, summary)

# GUI setup
root = tk.Tk()
root.title("Summarize-o-Matic")

# Set window size
root.geometry("1000x800")

# Text box for input
text_box = scrolledtext.ScrolledText(root, width=80, height=20, wrap=tk.WORD)
text_box.pack(pady=10)

# Button to summarize
summarize_button = tk.Button(root, text="Summarize", command=summarize_text)
summarize_button.pack(pady=5)

# Result box for output
result_box = scrolledtext.ScrolledText(root, width=80, height=20, wrap=tk.WORD)
result_box.pack(pady=10)

read_button = tk.Button(root, text="Read Out", command=read_sentence)
read_button.pack(pady=20)

root.mainloop()